<a href="https://colab.research.google.com/github/innovateDotAI/ReadyToUseModels/blob/main/Outlier_Detection_using_OneSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To use this notebook(Written in Python) it is assumed that the user has a basic understanding of Python. 
This Blog is  written as Jupyter notebooks and run directly in Google Colab—a hosted notebook environment that requires no setup. Click the Run in Google Colab button.

**Problem Statement**: We need to develop an outlier detection system which correctly label the minor class data as an outlier. We have used data set from UCI. For details click on link [AI4I 2020 Predictive Maintenance Dataset Data Set](https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset) . In this blog we are building classification models to predict the outlier from given dataset.
**Basic Desing approach**:We are using basic data mining concept of buidling outier classifier by makeing classsifieer for major calss recognition. If any data is not fitting in this model then that data will be treated as outlier. 
![picture](https://drive.google.com/uc?id=1z1Eueyk6_5cSVYLcSVyMWGIxpNov9ShZ)
In this Blog [sklearn.svm.OneClassSVM¶](https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html#sklearn.svm.OneClassSVM) used.

**Support Vector Machine basic concept**:Support Vector Machine (SVM) is one of the most popular Supervised Learning algorithms, which is used for Classification as well as Regression problems. However,primarily, it is used for Classification problems in Machine Learning. We use discriminative way of classifying the data. For discrimination Hyperplan is drawn between dataset of different class. All those points which fall on hyperplan is called support vectors. Our aim is to maximizing the margine of hyperplan. 

![picture](https://drive.google.com/uc?id=14ehTTnAMru6aBeMO2svWeE--C-of6Ta3)
We convert the maximization into a minimization problem which is defined as  Lagranges function. Since Lagranges function solution needs an inner product of all parts of a dataset with a support vector point which will be time consuming, so the kernel trick used which is some transfromatin function.The kernel function needs to satisfy the Mercer function, i.e., the function is “positive-definite”. This also means that optimization problem can be solved in **polynomial time**!. In our case we have used Radial basis function kernel (rbf) as hyper parameter.


For details on SVM algorithm please reffer to [Support Vector Machines for Machine Learning](https://machinelearningmastery.com/support-vector-machines-for-machine-learning/)

**Additoonal refrencees used:**
[One-Class Classification Algorithms for Imbalanced Datasets](https://machinelearningmastery.com/one-class-classification-algorithms/)

# 1. Load the Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
np.set_printoptions(precision=3, suppress=True)


# 2. Load the data

In [ ]:
#Data loading
#First download and import the dataset using pandas:
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv'
#column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                #'Acceleration', 'Model Year', 'Origin']
col_name = ['UID','PID','Type','airTemp','processTemp','rotationalSpeed','torque','toolwear','machineFailure','TWF','HDF','PWF','OSF','RNF']
raw_dataset = pd.read_csv(url, names=col_name,
                          na_values='?', comment='\t',
                          sep=',', skipinitialspace=True,header=0)

In [ ]:
raw_dataset.head()

,UID,PID,Type,airTemp,processTemp,rotationalSpeed,torque,toolwear,machineFailure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [ ]:
#Drop those rows which will not be part of feature vector:
rawData = raw_dataset.copy()
rawData.drop(['UID','PID','TWF','HDF','PWF','OSF','RNF'], axis = 1, inplace = True)
rawData.head()

,Type,airTemp,processTemp,rotationalSpeed,torque,toolwear,machineFailure
0,M,298.1,308.6,1551,42.8,0,0
1,L,298.2,308.7,1408,46.3,3,0
2,L,298.1,308.5,1498,49.4,5,0
3,L,298.2,308.6,1433,39.5,7,0
4,L,298.2,308.7,1408,40.0,9,0


# 3. Data Preprocessing 
## 3.1 Missing value investigation

In [ ]:
rawData.isna().sum()

Type               0
airTemp            0
processTemp        0
rotationalSpeed    0
torque             0
toolwear           0
machineFailure     0
dtype: int64

No missing value found

In [ ]:
# mark inliers 1, outliers -1
#rawSVMdata = rawData.copy()
# rawData['machineFailure' == '1'] = -1
# rawData['machineFailure' == '0'] = 1
# rawData
rawData.loc[rawData["machineFailure"] == 1, "machineFailure"] = -1
rawData.loc[rawData["machineFailure"] == 0, "machineFailure"] = 1

In [ ]:
rawData["machineFailure"].unique()

array([ 1, -1])

In [ ]:
rawData

,Type,airTemp,processTemp,rotationalSpeed,torque,toolwear,machineFailure
0,M,298.1,308.6,1551,42.8,0,1
1,L,298.2,308.7,1408,46.3,3,1
2,L,298.1,308.5,1498,49.4,5,1
3,L,298.2,308.6,1433,39.5,7,1
4,L,298.2,308.7,1408,40.0,9,1
...,...,...,...,...,...,...,...
9995,M,298.8,308.4,1604,29.5,14,1
9996,H,298.9,308.4,1632,31.8,17,1
9997,M,299.0,308.6,1645,33.4,22,1
9998,H,299.0,308.7,1408,48.5,25,1


## 3.2 One Hot encoding

In [ ]:
#raw_dataset
# # generate binary values using get_dummies
raw_dataset_oneHot = pd.get_dummies(rawData, columns=["Type"], prefix=["Type_is"] )
raw_dataset_oneHot

,airTemp,processTemp,rotationalSpeed,torque,toolwear,machineFailure,Type_is_H,Type_is_L,Type_is_M
0,298.1,308.6,1551,42.8,0,1,0,0,1
1,298.2,308.7,1408,46.3,3,1,0,1,0
2,298.1,308.5,1498,49.4,5,1,0,1,0
3,298.2,308.6,1433,39.5,7,1,0,1,0
4,298.2,308.7,1408,40.0,9,1,0,1,0
...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,1,0,0,1
9996,298.9,308.4,1632,31.8,17,1,1,0,0
9997,299.0,308.6,1645,33.4,22,1,0,0,1
9998,299.0,308.7,1408,48.5,25,1,1,0,0


In [ ]:
raw_dataset_oneHot["machineFailure"].unique()

array([ 1, -1])

## 3.3 Split the data based on class

In [ ]:
# Classs count 
#data.groupby('amount', as_index=False).agg({"duration": "sum"})
#raw_dataset_oneHot.groupby('machineFailure',as_index=False).agg({"machineFailure": "count"})
raw_dataset_oneHot.groupby('machineFailure',as_index=True).agg({"machineFailure": "count"})

,machineFailure
machineFailure,
-1,339
1,9661


Normal Classs is "machineFailure = 0" and outlier class is "machineFailure = 1"

In [ ]:
#split raw dataset into two part:
raw_dataset_oneHot_MFFalse = raw_dataset_oneHot[raw_dataset_oneHot['machineFailure'] == 1]
raw_dataset_oneHot_MFTrue = raw_dataset_oneHot[raw_dataset_oneHot['machineFailure'] == -1]

In [ ]:
raw_dataset_oneHot_MFFalse

,airTemp,processTemp,rotationalSpeed,torque,toolwear,machineFailure,Type_is_H,Type_is_L,Type_is_M
0,298.1,308.6,1551,42.8,0,1,0,0,1
1,298.2,308.7,1408,46.3,3,1,0,1,0
2,298.1,308.5,1498,49.4,5,1,0,1,0
3,298.2,308.6,1433,39.5,7,1,0,1,0
4,298.2,308.7,1408,40.0,9,1,0,1,0
...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,1,0,0,1
9996,298.9,308.4,1632,31.8,17,1,1,0,0
9997,299.0,308.6,1645,33.4,22,1,0,0,1
9998,299.0,308.7,1408,48.5,25,1,1,0,0


In [ ]:
# Understand data distribution
raw_dataset_oneHot_MFFalse.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
airTemp,9661.0,299.973999,1.990748,295.3,298.3,300.0,301.5,304.5
processTemp,9661.0,309.995570,1.486846,305.7,308.8,310.0,311.1,313.8
rotationalSpeed,9661.0,1540.260014,167.394734,1168.0,1429.0,1507.0,1615.0,2695.0
torque,9661.0,39.629655,9.472080,12.6,33.1,39.9,46.3,70.0
toolwear,9661.0,106.693717,62.945790,0.0,52.0,107.0,160.0,246.0
machineFailure,9661.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
Type_is_H,9661.0,0.101646,0.302198,0.0,0.0,0.0,0.0,1.0
Type_is_L,9661.0,0.596729,0.490580,0.0,0.0,1.0,1.0,1.0
Type_is_M,9661.0,0.301625,0.458987,0.0,0.0,0.0,1.0,1.0


## 3.4 Create Traning and test Set

In [ ]:
#Split the data into training and test sets
train_dataset = raw_dataset_oneHot_MFFalse.sample(frac=0.8, random_state=0)
test_dataset = raw_dataset_oneHot_MFFalse.drop(train_dataset.index) # Removing all rows which is part of traning set.

In [ ]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7729 entries, 1896 to 9964
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airTemp          7729 non-null   float64
 1   processTemp      7729 non-null   float64
 2   rotationalSpeed  7729 non-null   int64  
 3   torque           7729 non-null   float64
 4   toolwear         7729 non-null   int64  
 5   machineFailure   7729 non-null   int64  
 6   Type_is_H        7729 non-null   uint8  
 7   Type_is_L        7729 non-null   uint8  
 8   Type_is_M        7729 non-null   uint8  
dtypes: float64(3), int64(3), uint8(3)
memory usage: 445.3 KB


In [ ]:
#Split features from labels
#The pop() method removes the specified column from the DataFrame. The pop() method returns the removed columns as a Pandas Series object.
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('machineFailure') # Removing label from Traning_set data
test_labels = test_features.pop('machineFailure')

# 4. Normalization - 
## 4.1 Noraml Class: ['machineFailure'] == 0] ( N(0,1))

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = preprocessing.StandardScaler().fit(train_features.values)
X_scaled = scaler.transform(train_features.values)
print(X_scaled.mean(axis=0))
print(X_scaled.std(axis=0))
X_scaled

[-0. -0.  0.  0.  0. -0. -0.  0.]
[1. 1. 1. 1. 1. 1. 1. 1.]


array([[-1.04 , -1.681,  0.213, ..., -0.34 ,  0.827, -0.659],
       [ 1.318,  1.548,  0.165, ..., -0.34 , -1.209,  1.518],
       [-0.889, -1.479, -0.985, ..., -0.34 ,  0.827, -0.659],
       ...,
       [-0.087, -0.537, -0.6  , ..., -0.34 ,  0.827, -0.659],
       [ 0.365,  0.673, -1.22 , ..., -0.34 ,  0.827, -0.659],
       [-0.839, -1.412,  1.37 , ..., -0.34 ,  0.827, -0.659]])

In [ ]:
X_scaled.shape

(7729, 8)

## 4.2 Anomaly Class: ['machineFailure'] == 1] ( N(0,1)) 

In [ ]:
raw_dataset_oneHot_MFTrue

,airTemp,processTemp,rotationalSpeed,torque,toolwear,machineFailure,Type_is_H,Type_is_L,Type_is_M
50,298.9,309.1,2861,4.6,143,-1,0,1,0
69,298.9,309.0,1410,65.7,191,-1,0,1,0
77,298.8,308.9,1455,41.3,208,-1,0,1,0
160,298.4,308.2,1282,60.7,216,-1,0,1,0
161,298.3,308.1,1412,52.3,218,-1,0,1,0
...,...,...,...,...,...,...,...,...,...
9758,298.6,309.8,2271,16.2,218,-1,0,1,0
9764,298.5,309.5,1294,66.7,12,-1,0,1,0
9822,298.5,309.4,1360,60.9,187,-1,0,1,0
9830,298.3,309.3,1337,56.1,206,-1,0,1,0


In [ ]:
#Split features from labels
#The pop() method removes the specified column from the DataFrame. The pop() method returns the removed columns as a Pandas Series object.
ana_features = raw_dataset_oneHot_MFTrue.copy()
ana_labels = ana_features.pop('machineFailure') # Removing label from Traning_set data

In [ ]:
ana_features.shape

(339, 8)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = preprocessing.StandardScaler().fit(train_features.values)
X_anomaly= scaler.transform(ana_features.values)
print(X_anomaly.mean(axis=0))
print(X_anomaly.std(axis=0))
X_anomaly

[ 0.458  0.196 -0.253  1.106  0.58  -0.137  0.203 -0.126]
[1.038 0.916 2.315 1.731 1.15  0.79  0.939 0.936]


array([[-0.538, -0.605,  7.963, ..., -0.34 ,  0.827, -0.659],
       [-0.538, -0.672, -0.774, ..., -0.34 ,  0.827, -0.659],
       [-0.588, -0.739, -0.503, ..., -0.34 ,  0.827, -0.659],
       ...,
       [-0.739, -0.403, -1.075, ..., -0.34 ,  0.827, -0.659],
       [-0.839, -0.47 , -1.214, ..., -0.34 ,  0.827, -0.659],
       [-0.689, -1.21 , -1.069, ..., -0.34 ,  0.827, -0.659]])

# 5. Normal classs recognition Model Creation and Prediction 
## 5.1 Create OneClassSVM model

In [ ]:
from sklearn.svm import OneClassSVM
# define outlier detection model
model = OneClassSVM(gamma='scale', nu=0.9)
#model =OneClassSVM(kernel='poly', degree=2, gamma='scale', coef0=0.0, tol=0.001, nu=0.01, shrinking=True, cache_size=200, verbose=False, max_iter=- 1)

In [ ]:
# fit on majority class
trainX= X_scaled
model.fit(trainX)

OneClassSVM(nu=0.9)

In [ ]:
# detect outliers in the test set
yhat_train = model.predict(X_scaled)
print(yhat_train)

[-1 -1 -1 ... -1 -1 -1]


## 5.2 Confusion matirx of  outlier data for analysis

In [ ]:
# #trainX = trainX[trainy==0]
# trainX= X_scaled
# model.fit(trainX)
# # detect outliers in the test set
yhat_ana = model.predict(X_anomaly)
print(yhat_ana)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [ ]:
#Check the classification model strength
from sklearn.metrics import classification_report
target_names = ['-1','1'] # Need to automate this step later
y_ana = np.array(ana_labels)
y_predict_ana = yhat_ana.astype('int64')
#print(classification_report(np.array(test_labels.astype('float32'))), test_predictions, target_names=target_names))
print(classification_report(y_ana, y_predict_ana, target_names=target_names))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       339
           1       0.00      0.00      0.00         0

    accuracy                           1.00       339
   macro avg       0.50      0.50      0.50       339
weighted avg       1.00      1.00      1.00       339



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Conclusion
By using oneSVM we are able to seperate outluer data successfully with 100% accuracy. This is achived by hyper parameter tuning. There is no one solution fit for all data. So we need to check various outlier models before adopting any model. 

**Author: Bipin Kumar sinha**

**InnovateDotAI**